In [1]:
import logging

import numpy as np
import optimistix as optx
from atmodeller import (
    InteriorAtmosphere,
    Planet,
    Species,
    SpeciesCollection,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
    SolverParameters,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm

from scipy.interpolate import interp1d
import os

from matplotlib.ticker import ScalarFormatter

formatter = ScalarFormatter(useMathText=False)
formatter.set_scientific(False)
formatter.set_useOffset(False)
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

logger = debug_logger() #you can dump it in file 
#logger.setLevel(logging.INFO) #you could set it to error

logger.setLevel(logging.DEBUG)  
##rebase for atmodeller git pull, git re base (you might also have to git pull when paralell)

Atmodeller initialized with double precision (float64)


In [2]:
#Setting up the different layers for Uranus 

#Overall structure 
R_earth = 6.371e6
R_uranus = 2.5388e7 #meters
M_earth = 5.972e24 #kg
M_uranus = 14.536*M_earth

#Boundary inner and outer envelope
boundary_mass = 0.913*M_uranus 
boundary_radius = 0.772*R_uranus
boundary_temperature = 2338.8
boundary_core_mass_fraction = 0
boundary_mantle_melt_fraction = 0

#Core 
core_mass = 0.61*M_earth 
core_mass_fraction_U = 0
core_mantle_melt_fraction = 1.0
core_radius = 0.7*R_earth 
core_temperature = 6000

Z_1 = 0.17
Z_2 = 0.915
Z_solar = 0.0142
RANDOM_SEED = 0

np.random.seed(RANDOM_SEED)

In [3]:
H_masspercent_lodders09: float = 73.9
He_masspercent_lodders09: float = 24.69
C_masspercent_lodders09: float = 0.22
N_masspercent_lodders09: float = 0.07
O_masspercent_lodders09: float = 0.63
Si_masspercent_lodders09: float = 0.07
Mg_masspercent_lodders09: float = 0.06

H_logN: float = 12
He_logN: float = 10.93
C_logN: float = 8.39
N_logN: float = 7.86
O_logN: float = 8.73
Si_logN: float = 7.53
Mg_logN: float = 7.54 

# Calculating Boundary Core and Inner Envelope

In [7]:
#SiO2 for primitive composition of Earth
SiO2_mantlemasspercent_palme14: float = 45.4
total_mantlemasspercent_palme14: float = 98.41
Si_massfraction_palme14: float = round(
    SiO2_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (28.0855 + 2 * 15.999)
    * 28.0855,
    3,
)

O_massfraction_palme141: float = Si_massfraction_palme14 / 28.0855 * 2 * 15.999

MgO_mantlemasspercent_palme14: float = 36.77
Mg_massfraction_palme14: float = round(
    MgO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (24.305 + 15.999)
    * 24.305,
    3,
)

FeO_mantlemasspercent_palme14: float = 8.10
Fe_massfraction_palme14: float = round(
    FeO_mantlemasspercent_palme14
    / total_mantlemasspercent_palme14
    * (1 - core_mass_fraction_U)
    / (55.845 + 15.999)
    * 55.845,
    3,
)

O_massfraction_palme142: float = Mg_massfraction_palme14 / 24.305 * 15.999
O_mass_fraction_palme143: float = Fe_massfraction_palme14 / 55.845 * 15.999
O_massfraction_palme14 = O_massfraction_palme141 #+ O_massfraction_palme142
print(O_massfraction_palme14)

0.24609026009862742


In [9]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

H2O_g = Species.create_gas("H2O")
H2O_gs = Species.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = Species.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = Species.create_gas("H2")
H2_gs = Species.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = Species.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = Species.create_gas("O2")
O2_rg = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = Species.create_gas("OSi")
OSi_rg = Species.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = Species.create_gas("H4Si")
H4Si_rg = Species.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = Species.create_condensed("O2Si", state="l")
O2Si_bqz = Species.create_condensed("O2Si", state="bqz")

CO2_g = Species.create_gas("CO2")
CO2_gs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = Species.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)
#
CO_g = Species.create_gas("CO")
CO_gs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = Species.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)
#
CH4_g = Species.create_gas("CH4")
CH4_gs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = Species.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)
#
N2_g = Species.create_gas("N2")
N2_gs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = Species.create_gas(
    "N2", solubility=sol_models["N2_basalt_libourel03"], activity=eos_models["N2_cs_saxena87"]
)

NH3_g = Species.create_gas("NH3")
NH3_rg = Species.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = Species.create_gas("He")
He_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = Species.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

Mg_g = Species.create_gas("Mg")
#Mg_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
#Mg_rgs = Species.create_gas(
#    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
#)

MgO_l = Species.create_condensed("MgO", state="l")
MgO_cr = Species.create_condensed("MgO", state="cr")
MgH_g = Species.create_gas("MgH")
MgOH_g = Species.create_gas("MgOH")

species_HHeCNOSi_magma_nosol_ideal = SpeciesCollection(
    (H2O_g, H2_g, O2_g, OSi_g, H4Si_g, O2Si_l , He_g) #CO2_g, CO_g, CH4_g, N2_g, NH3_g,, Mg_g, MgO_l, MgH_g, MgOH_g)
     #)
)
species_HHeCNOSi_magma_sol_ideal = SpeciesCollection(
    (H2O_gs, H2_gs, O2_g, OSi_g, H4Si_g, O2Si_l ,  He_gs)#CO2_gs, CO_gs, CH4_gs, N2_gs, NH3_g,, Mg_g, MgO_l, MgH_g, MgOH_g)
     #, )
)
species_HHeCNOSi_magma_sol_real = SpeciesCollection(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
        H4Si_rg,
        O2Si_l,
#        CO2_rgs,
#        CO_rgs,
#        CH4_rgs,
#        N2_rgs,
#        NH3_rg,
        He_rgs,
#        Mg_g, 
#        MgO_l,
#        MgH_g,
#        MgOH_g
    )
)
#MgH Magnesium Hydride
#MgOH Magnesium Hydroxide -> ideal gas law 
#Plot condensed moles for example for Silicon
#mass balance, take mass constraints, take atmospheric mass/moles+ take condensed mass/moles + take dissolved mass/moles
#plot envelope mass fraction, envelope mass/planet mass, Helled, what do they call the volatile layer. 
#temperature-pressure profile at 1 Myr or 10 Myr
#Ravit Helled 2011-2013

In [6]:
T_resolution = 20
H_resolution = 20 
#flatten look for numpy mesh, all condensates seperately
temperature = np.repeat(np.linspace(5500, 6500, T_resolution, endpoint=False), H_resolution)
temperature = temperature.flatten()
hmps = np.tile(np.logspace(-1, 0.5, num=H_resolution), int(np.ceil(T_resolution)))[:T_resolution*H_resolution]
hmps = hmps.flatten()

metall = 1

In [12]:
#T_resolution = 2
#H_resolution = 1 
#flatten look for numates seperately
#temperature = np.repeat(np.linspace(6000,6000, T_resolution, endpoint=False), H_resolution)
#temperature = temperature.flatten()
#hmps = np.tile(np.logspace(-1,-1, num=H_resolution), int(np.ceil(T_resolution)))[:T_resolution*H_resolution]
#hmps = hmps.flatten()
#hmps = np.logspace(-4,-2, num=20)
#hmps = np.logspace(-4,-2, num=20)
hmps = 1
temperature = 4500
metall = 1
h_kgs = hmps / 100 * boundary_mass
si_kg_magma: float = Si_massfraction_palme14 * core_mass
mg_kg_magma: float = Mg_massfraction_palme14 * core_mass
o_kg_magma: float = O_massfraction_palme14 * core_mass

# Lodders et al. (2009) Springer book chapter Table 8
si_kgs_solar = h_kgs * Si_masspercent_lodders09 / H_masspercent_lodders09
mg_kgs_solar = h_kgs * Mg_masspercent_lodders09 / H_masspercent_lodders09
o_kgs_solar = h_kgs * O_masspercent_lodders09 / H_masspercent_lodders09
c_kgs_solar = h_kgs * C_masspercent_lodders09 / H_masspercent_lodders09
n_kgs_solar = h_kgs * N_masspercent_lodders09 / H_masspercent_lodders09
he_kgs_solar = h_kgs * He_masspercent_lodders09 / H_masspercent_lodders09

mass_constraints = {
    "H": h_kgs,
    "He": he_kgs_solar,
#    "C": metall * c_kgs_solar,
#    "N": metall * n_kgs_solar,
    "Si": metall * si_kgs_solar + core_mantle_melt_fraction * si_kg_magma,
    "O": metall * o_kgs_solar +core_mantle_melt_fraction * o_kg_magma,
#    "Mg": metall * mg_kgs_solar + core_mantle_melt_fraction * mg_kg_magma,
        }
solver_parameters = SolverParameters(multistart= 30)
Uranus = Planet(
    surface_temperature=core_temperature,
    planet_mass=core_mass,
    mantle_melt_fraction=core_mantle_melt_fraction,
    core_mass_fraction = core_mass_fraction_U,
    surface_radius=core_radius,
    )

model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_density = output_magma_nosol_ideal.log_number_density
model_magma_sol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")

#last minute solution, reverting to other solver 
model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        solver_parameters= solver_parameters,
        initial_log_number_density=initial_log_number_density
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("Core_real_sol_new_4500")

[18:03:38 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: IdealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'O2Si_l: CondensateActivity, NoSolubility', 'He_g: IdealGas, NoSolubility')
[18:03:38 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 H2O_g + 1.0 OSi_g = 1.5 O2_g + 1.0 H4Si_g',
 1: '0.5 O2_g + 1.0 OSi_g = 1.0 O2Si_l',
 2: '1.0 H2O_g = 1.0 H2_g + 0.5 O2_g'}
[18:03:38 - atmodeller.classes             - INFO     ] - Attempting to solve 1 model(s)
[18:03:38 - atmodeller.classes             - INFO     ] - Solve complete: 1 (100.00%) successful model(s)
[18:03:38 - atmodeller.classes             - INFO     ] - Solver steps (max) = 40
[18:03:38 - atmodeller.output              - DEBUG    ] - Creating Output
[18:03:38 - atmodeller.output              - INFO     ] - Writing output to excel
[18:03:38 - atmodeller.output              - INFO     ]

# Calculating boundary inner and outer envelope

In [4]:
filename = "Core_real_sol_001.xlsx"

In [5]:
h_kg_inner = pd.read_excel(filename, sheet_name="element_H")["atmosphere_mass"]
o_kg_inner = pd.read_excel(filename, sheet_name="element_O")["atmosphere_mass"]
si_kg_inner = pd.read_excel(filename, sheet_name="element_Si")["atmosphere_mass"]
c_kg_inner = pd.read_excel(filename, sheet_name="element_C")["atmosphere_mass"]
he_kg_inner = pd.read_excel(filename, sheet_name="element_He")["atmosphere_mass"]
n_kg_inner = pd.read_excel(filename, sheet_name="element_N")["atmosphere_mass"]
mg_kg_inner = pd.read_excel(filename, sheet_name="element_Mg")["atmosphere_mass"]

In [19]:
eos_models = get_eos_models()
sol_models = get_solubility_models()

#you can use other condensates - ices important hold off for the moment, graphite, you could always add some
H2O_g = Species.create_gas("H2O")
H2O_gs = Species.create_gas("H2O", solubility=sol_models["H2O_peridotite_sossi23"])
H2O_rgs = Species.create_gas(
    "H2O",
    activity=eos_models["H2O_cork_holland98"],
    solubility=sol_models["H2O_peridotite_sossi23"],
)

H2_g = Species.create_gas("H2")
H2_gs = Species.create_gas("H2", solubility=sol_models["H2_basalt_hirschmann12"])
H2_rgs = Species.create_gas(
    "H2", activity=eos_models["H2_chabrier21"], solubility=sol_models["H2_basalt_hirschmann12"]
)

O2_g = Species.create_gas("O2")
O2_rg = Species.create_gas("O2", activity=eos_models["O2_cs_shi92"])

OSi_g = Species.create_gas("OSi")
OSi_rg = Species.create_gas("OSi", activity=eos_models["OSi_rk49_connolly16"])

H4Si_g = Species.create_gas("H4Si")
H4Si_rg = Species.create_gas("H4Si", activity=eos_models["H4Si_wang18"])

O2Si_l = Species.create_condensed("O2Si", state="l")
#O2Si_bqz = Species.create_condensed("O2Si", state="bqz")

CO2_g = Species.create_gas("CO2")
CO2_gs = Species.create_gas("CO2", solubility=sol_models["CO2_basalt_dixon95"])
CO2_rgs = Species.create_gas(
    "CO2", solubility=sol_models["CO2_basalt_dixon95"], activity=eos_models["CO2_cs_shi92"]
)
#
CO_g = Species.create_gas("CO")
CO_gs = Species.create_gas("CO", solubility=sol_models["CO_basalt_yoshioka19"])
CO_rgs = Species.create_gas(
    "CO", solubility=sol_models["CO_basalt_yoshioka19"], activity=eos_models["CO_cs_shi92"]
)
#
CH4_g = Species.create_gas("CH4")
CH4_gs = Species.create_gas("CH4", solubility=sol_models["CH4_basalt_ardia13"])
CH4_rgs = Species.create_gas(
    "CH4", solubility=sol_models["CH4_basalt_ardia13"], activity=eos_models["CH4_cs_shi92"]
)
#
N2_g = Species.create_gas("N2")
N2_gs = Species.create_gas("N2", solubility=sol_models["N2_basalt_libourel03"])
N2_rgs = Species.create_gas(
    "N2", solubility=sol_models["N2_basalt_libourel03"], activity=eos_models["N2_cs_saxena87"]
)

NH3_g = Species.create_gas("NH3")
NH3_rg = Species.create_gas("H3N", activity=eos_models["H3N_rk49_reid87"])

He_g = Species.create_gas("He")
He_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
He_rgs = Species.create_gas(
    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
)

Mg_g = Species.create_gas("Mg")
#Mg_gs = Species.create_gas("He", solubility=sol_models["He_basalt_jambon86"])
#Mg_rgs = Species.create_gas(
#    "He", solubility=sol_models["He_basalt_jambon86"], activity=eos_models["He_chabrier21"]
#)

MgO_l = Species.create_condensed("MgO", state="l")
#MgO_cr = Species.create_condensed("MgO", state="cr")
MgH_g = Species.create_gas("MgH")
MgOH_g = Species.create_gas("MgOH")

species_HHeCNOSi_magma_nosol_ideal = SpeciesCollection(
    (H2O_g, H2_g, O2_g, OSi_rg, H4Si_g, CO2_g, CO_g, CH4_g, N2_g, NH3_g, He_g, Mg_g, MgH_g, MgOH_g)
     #)
)
#solubility not needed
species_HHeCNOSi_magma_sol_ideal = SpeciesCollection(
    (H2O_gs, H2_gs, O2_g, OSi_rg, H4Si_g,CO2_gs, CO_gs, CH4_gs, N2_gs, NH3_g, He_gs, Mg_g, MgH_g, MgOH_g)
     #, )
)
species_HHeCNOSi_magma_sol_real = SpeciesCollection(
    (
        H2O_rgs,
        H2_rgs,
        O2_rg,
        OSi_rg,
#        O2Si_l,
        H4Si_rg,
        CO2_rgs,
        CO_rgs,
        CH4_rgs,
        N2_rgs,
        NH3_rg,
        He_rgs,
        Mg_g, 
        MgH_g,
        MgOH_g,
#        MgO_l
    )
)
#MgH Magnesium Hydride
#MgOH Magnesium Hydroxide -> ideal gas law 
#Plot condensed moles for example for Silicon
#mass balance, take mass constraints, take atmospheric mass/moles+ take condensed mass/moles + take dissolved mass/moles
#plot envelope mass fraction, envelope mass/planet mass, Helled, what do they call the volatile layer. 
#temperature-pressure profile at 1 Myr or 10 Myr
#Ravit Helled 2011-2013

In [26]:
temperature = boundary_temperature
metall = Z_1/Z_solar
hmps = 1
h_kgs = hmps / 100 * M_uranus
# Lodders et al. (2009) Springer book chapter Table 8
si_kgs_solar = h_kgs * Si_masspercent_lodders09 / H_masspercent_lodders09
mg_kgs_solar = h_kgs * Mg_masspercent_lodders09 / H_masspercent_lodders09
o_kgs_solar = h_kgs * O_masspercent_lodders09 / H_masspercent_lodders09
c_kgs_solar = h_kgs * C_masspercent_lodders09 / H_masspercent_lodders09
n_kgs_solar = h_kgs * N_masspercent_lodders09 / H_masspercent_lodders09
he_kgs_solar = h_kgs * He_masspercent_lodders09 / H_masspercent_lodders09

mass_constraints = {
    "H": h_kg_inner + h_kgs,
    "He": he_kg_inner + he_kgs_solar,
    "C": metall * c_kgs_solar + c_kg_inner,
    "N": metall * n_kgs_solar + n_kg_inner,
    "Si": metall * si_kgs_solar + si_kg_inner,
    "O": metall * o_kgs_solar + o_kg_inner,
    "Mg": metall * mg_kgs_solar + mg_kg_inner,
        }

In [27]:
Uranus = Planet(
    surface_temperature=temperature,
    planet_mass=boundary_mass,
    core_mass_fraction = boundary_core_mass_fraction,
    surface_radius=boundary_radius,
    )

model_magma_nosol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_nosol_ideal)
model_magma_nosol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        )
output_magma_nosol_ideal = model_magma_nosol_ideal.output
output_magma_nosol_ideal.quick_look()
output_magma_nosol_ideal.to_excel("HHeCNOSi_magma_nosol_ideal")

model_magma_sol_ideal = InteriorAtmosphere(species_HHeCNOSi_magma_sol_ideal)
initial_log_number_density = output_magma_nosol_ideal.log_number_density
model_magma_sol_ideal.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        initial_log_number_density=initial_log_number_density,
        )
output_magma_sol_ideal = model_magma_sol_ideal.output
output_magma_sol_ideal.quick_look()
output_magma_sol_ideal.to_excel("HHeCNOSi_magma_sol_ideal")
solver_parameters = SolverParameters(multistart= 20)
#last minute solution, reverting to other solver 
model_magma_sol_real = InteriorAtmosphere(species_HHeCNOSi_magma_sol_real)
initial_log_number_density = output_magma_sol_ideal.log_number_density
model_magma_sol_real.solve(
        planet=Uranus,
        mass_constraints=mass_constraints,
        solver_parameters= solver_parameters,
        initial_log_number_density=initial_log_number_density
        )
output_magma_sol_real = model_magma_sol_real.output
output_magma_sol_real.quick_look()
output_magma_sol_real.to_excel("Envelope_real_sol_0001")

[14:23:54 - atmodeller.classes             - INFO     ] - species = ('H2O_g: IdealGas, NoSolubility', 'H2_g: IdealGas, NoSolubility', 'O2_g: IdealGas, NoSolubility', 'OSi_g: CombinedRealGas, NoSolubility', 'H4Si_g: IdealGas, NoSolubility', 'CO2_g: IdealGas, NoSolubility', 'CO_g: IdealGas, NoSolubility', 'CH4_g: IdealGas, NoSolubility', 'N2_g: IdealGas, NoSolubility', 'H3N_g: IdealGas, NoSolubility', 'He_g: IdealGas, NoSolubility', 'Mg_g: IdealGas, NoSolubility', 'HMg_g: IdealGas, NoSolubility', 'HMgO_g: IdealGas, NoSolubility')
[14:23:54 - atmodeller.classes             - INFO     ] - reactions = {0: '4.0 H2_g + 1.0 CO2_g = 2.0 H2O_g + 1.0 CH4_g',
 1: '1.0 H2_g + 1.0 CO2_g = 1.0 H2O_g + 1.0 CO_g',
 2: '1.5 H2_g + 0.5 N2_g = 1.0 H3N_g',
 3: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g',
 4: '1.0 H2O_g + 1.0 H4Si_g = 3.0 H2_g + 1.0 OSi_g',
 5: '0.5 H2_g + 1.0 Mg_g = 1.0 HMg_g',
 6: '1.0 H2O_g + 1.0 Mg_g = 0.5 H2_g + 1.0 HMgO_g'}
[14:23:54 - atmodeller.classes             - INFO     ] - Attempting to